In [ ]:
!pip uninstall -y transformers accelerate evaluate rouge_score
!pip install transformers accelerate evaluate rouge_score
!pip install sacrebleu
!pip install meteor


In [ ]:
# pip install wandb

In [ ]:
# import wandb

# wandb.init(project="qagpt2")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# 7ae06a5af942cfd2e21de64ee697126eb8b0d0b9

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import evaluate
import tensorflow as tf
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader

from evaluate import load
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments


In [ ]:
df_train = pd.read_excel("/content/drive/MyDrive/QA dataset talentsprint/aiml-qa-train.xlsx")
df_train.head()

In [ ]:
df_dev = pd.read_excel("/content/drive/MyDrive/QA dataset talentsprint/aiml-qa-dev.xlsx")
df_dev.head()

In [ ]:
list_of_references = []
for i in range(0, len(df)):
    list_of_references.append([df.iloc[i, 1], df.iloc[i, 2]])
print(len(list_of_references))
print(len(df))

In [ ]:
question_word_count_list = []
for sentence in df_train['question']:
  question_word_count_list.append(len(sentence.split(" ")))

answer_word_count_list = []
for sentence in df_train['answer']:
  answer_word_count_list.append(len(sentence.split(" ")))


In [ ]:
plt.hist(question_word_count_list, bins=10, color='blue')
plt.title('Question word count')
plt.xlabel('question length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.hist(answer_word_count_list, bins=10, color='blue')
plt.title('Answer word count')
plt.xlabel('Answer length')
plt.ylabel('Frequency')
plt.show()

In [ ]:
pip install bert_score

In [ ]:
train_file_path = "/content/drive/MyDrive/QA dataset talentsprint/question-answer-train.txt"
eval_file_path = "/content/drive/MyDrive/QA dataset talentsprint/question-answer-dev.txt"
model_name = 'gpt2'
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")
sacrebleu = evaluate.load("sacrebleu")
#meteor = evaluate.load('meteor')
output_dir = '/content/result2'
overwrite_output_dir = False
per_device_train_batch_size = 4
num_train_epochs = 20
save_steps = 10000

In [ ]:
def load_dataset(file_path, tokenizer):
    dataset = LineByLineTextDataset(
                tokenizer=tokenizer,
                file_path=file_path,
                block_size=512
    )

    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm
    )

    return data_collator


def postprocess_text(preds, labels):
    # TODO: Separate only the subject from string
    # Ensure that for preds, you have a list of only the generated subject parts
    # For labels, it should be a list of list of only the reference subjects
    # NO OTHER CONTENT: EMAIL / SEPARATORS SHOULD BE OUTPUT AFTER POSTPROCESSING

    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    # print('logits:', logits.shape)
    pred_ids = torch.argmax(logits, dim=-1)
    # print('pred_ids:', pred_ids.shape)

    return pred_ids, labels


def compute_metrics(eval_preds):
    list_of_prediction = []
    list_of_references = []
    for i in range(0, len(df)):
        try:
#               print(i)
#               print(list_of_references[i])
              inputs = tokenizer("Question: " + df.iloc[i, 0] + ' Answer: ', return_tensors="pt")
              inputs['input_ids'] = inputs['input_ids'].cpu()  # Move input tensor to CPU if necessary
              device = torch.device("cuda:0")  # Specify the CUDA device
              model.to(device)  # Move the model to the CUDA device

              # Move the input tensor to the CUDA device
              inputs['input_ids'] = inputs['input_ids'].to(device)
              outputs = model.generate(inputs['input_ids'], max_new_tokens=30, do_sample=True, top_k=30, top_p=0.95)
              prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
              # Generate outputs using the model on the CUDA device
              #print(prediction)
              prediction = prediction.split('Answer: ')[1]
              list_of_prediction.append(prediction)
              list_of_references.append([df.iloc[i, 1], df.iloc[i, 2]])
              i = i + 1
#               print(prediction)
        except Exception as e:
              i = i + 1
    result = rouge.compute(predictions=list_of_prediction, references=list_of_references)
    results_sacrebleu = sacrebleu.compute(predictions=list_of_prediction, references=list_of_references, lowercase = True)
    results_bert = bertscore.compute(predictions=list_of_prediction, references=list_of_references, lang="en")
    #results_meteor = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    wandb.log({
        "R1": round(result["rouge1"], 4),
        "R2": round(result["rouge2"], 4),
        "RL": round(result["rougeL"], 4),
        "RLsum": round(result["rougeLsum"], 4),
        "bleu": round(results_sacrebleu["score"], 4),
        "precision1": round(results_bert["precision"][0], 4),
        "precision2": round(results_bert["precision"][1], 4),
        "recall1": round(results_bert["recall"][0], 4),
        "recall2": round(results_bert["recall"][1], 4),
        "f1-score1": round(results_bert["f1"][0], 4),
        "f1-score2": round(results_bert["f1"][1], 4)
    })
    return {
        "R1": round(result["rouge1"], 4),
        "R2": round(result["rouge2"], 4),
        "RL": round(result["rougeL"], 4),
        "RLsum": round(result["rougeLsum"], 4),
        "bleu": round(results_sacrebleu["score"], 4),
        "precision1":round(results_bert["precision"][0], 4),
        "precision2":round(results_bert["precision"][1], 4),
        "recall1":round(results_bert["recall"][0], 4),
        "recall2":round(results_bert["recall"][1], 4),
        "f1-score1":round(results_bert["f1"][0], 4),
        "f1-score2":round(results_bert["f1"][1], 4)
    }


tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.save_pretrained(output_dir)

train_dataset = load_dataset(train_file_path, tokenizer)
eval_dataset = load_dataset(eval_file_path, tokenizer)
data_collator = load_data_collator(tokenizer)

tokenizer.save_pretrained(output_dir)

training_args = TrainingArguments(
          output_dir=output_dir,
          evaluation_strategy = "epoch",
#          eval_steps = 500,
          learning_rate=1e-4,
          save_strategy = "steps",
          save_steps = 10000,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          per_device_eval_batch_size=1,
          num_train_epochs=num_train_epochs
      )

trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          preprocess_logits_for_metrics=preprocess_logits_for_metrics,
          compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model()